# Задание по программированию: Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = []

with open('checkins.dat') as file:
    column_types = list(map(lambda x: x.strip(), file.readline().strip().split('|')))
    for item in file:
        values = list(map(lambda x: x.strip(), item.strip().split('|')))
        if len(values) == len(column_types):
            data.append(values)

In [3]:
data_df = pd.DataFrame(data)
data_df.columns = column_types
data_df = data_df[(data_df.longitude != '') & (data_df.latitude != '')]
print(data_df.head())

        id  user_id venue_id    latitude     longitude           created_at
1   984222    15824     5222  38.8951118   -77.0363658  2012-04-21 17:43:47
3   984234    44652     5222   33.800745     -84.41052  2012-04-21 17:43:43
7   984291   105054     5222  45.5234515  -122.6762071  2012-04-21 17:39:22
9   984318  2146539     5222   40.764462   -111.904565  2012-04-21 17:35:46
10  984232    93870   380645  33.4483771  -112.0740373  2012-04-21 17:38:18


In [4]:
data_df.to_csv('data.csv')

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [5]:
coordinates = data_df[['latitude', 'longitude']]

In [6]:
coordinates = coordinates[:100000]

In [7]:
print(coordinates)

          latitude     longitude
1       38.8951118   -77.0363658
3        33.800745     -84.41052
7       45.5234515  -122.6762071
9        40.764462   -111.904565
10      33.4483771  -112.0740373
...            ...           ...
233788      33.575  -117.7255556
233789   37.629349   -122.400087
233793  29.7628844   -95.3830615
233797   32.802955    -96.769923
233798  37.7749295  -122.4194155

[100000 rows x 2 columns]


In [8]:
from sklearn.cluster import MeanShift

In [9]:
model = MeanShift(bandwidth=0.1)
model.fit(coordinates)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
          n_jobs=None, seeds=None)

In [10]:
coordinates['cluster'] = model.predict(coordinates)

In [11]:
print(coordinates)

          latitude     longitude  cluster
1       38.8951118   -77.0363658        5
3        33.800745     -84.41052        7
7       45.5234515  -122.6762071       30
9        40.764462   -111.904565       65
10      33.4483771  -112.0740373        1
...            ...           ...      ...
233788      33.575  -117.7255556       50
233789   37.629349   -122.400087        4
233793  29.7628844   -95.3830615       25
233797   32.802955    -96.769923       19
233798  37.7749295  -122.4194155        4

[100000 rows x 3 columns]


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [12]:
cluster_size = pd.DataFrame(coordinates.pivot_table(index = 'cluster', aggfunc = 'count', values = 'longitude'))
cluster_size.columns = ['clust_size']
print(cluster_size)

         clust_size
cluster            
0             12506
1              4692
2              3994
3              3363
4              3526
...             ...
3226              1
3227              1
3228              1
3229              1
3230              1

[3231 rows x 1 columns]


In [13]:
cluster_centers = pd.DataFrame(model.cluster_centers_)
cluster_centers.columns = ['center_latitude', 'center_longitude']
cluster_centers = cluster_centers.join(cluster_size)
cluster_centers = cluster_centers[cluster_centers.clust_size > 15]
print(cluster_centers.head())

   center_latitude  center_longitude  clust_size
0        40.717716        -73.991835       12506
1        33.449438       -112.002140        4692
2        33.446380       -111.901888        3994
3        41.878244        -87.629843        3363
4        37.688682       -122.409330        3526


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов.

In [14]:
offices_coordinates = [(33.751277, -118.188740), (25.867736, -80.324116), (51.503016, -0.075479),
                       (52.378894, 4.885084), (39.366487, 117.036146), (-33.868457, 151.205134)]

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [15]:
def distance(cl_lat, cl_long, off_lat, off_long):
    return ((cl_lat - off_lat) ** 2 + (cl_long - off_long) ** 2) ** 0.5

In [16]:
def dist_to_nearest_office(cl_lat, cl_long):
    distances = np.array([])
    for office in offices_coordinates:
        distances = np.append(distances, distance(cl_lat, cl_long, office[0], office[1]))
    return distances.min()

In [17]:
cluster_centers['nearest_office_dist'] = list(map(dist_to_nearest_office, cluster_centers.center_latitude,
                                                  cluster_centers.center_longitude))

In [18]:
cluster_centers = cluster_centers.sort_values(by = ['nearest_office_dist'])

In [19]:
cluster_centers

,center_latitude,center_longitude,clust_size,nearest_office_dist
420,-33.860630,151.204776,28,0.007835
370,52.372964,4.892317,31,0.009353
419,25.845672,-80.318891,28,0.022674
58,51.502991,-0.125537,254,0.050058
51,33.809878,-118.148924,281,0.070848
...,...,...,...,...
105,-23.549518,-46.638219,116,59.806393
433,-33.425741,-70.570740,27,60.090305
262,-22.903539,-43.209587,47,61.287238
103,-0.301226,36.522964,120,61.450391


In [20]:
with open('answer_cluster_banner.txt', 'w') as fout:
    fout.write(str(cluster_centers.iloc[0]['center_latitude']) + ' ' + 
               str(cluster_centers.iloc[0]['center_longitude']))